# Parallelization for beginners


In this notebook, I would like to demonstrate the power of parallelizing code and how easy it is to do that in Python. 

In [136]:
# Some libraries that we would need for parallel processing
import multiprocessing
from joblib import Parallel, delayed

# library to time the running time of the code
import time

# numpy library for the 
import numpy as np

The idea is to generate 1 million random numbers and calculate their mean and standard deviations. To see the efficiency of parallelizing, we will repeat this process 1000 times. 

We will use the **random.normal** function from the numpy library and generate a sample of size 1 million from the standard normal distribution.

This function takes three arguments: 

* loc : This is the mean of the normal distribution, we set it equal to 0 since we want to draw from a standard normal distribution 

* loc : This is the standard deviation of the normal distribution. We set it equal to 1. 

* size : It is the size of the sample that we want. 

To see a demonstration of this function, let's draw a sample of size 1 million from a standard normal distribution. 

In [137]:
np.random.normal(loc = 0, scale = 1, size = 1000000)


array([-1.03826774,  0.83501585, -0.8113014 , ..., -0.6735688 ,
       -0.25864818, -1.06519873])

Now let's write a simple function which takes an input `i` and sets the random seed equal to `i`, for the purpose of reproducibility. It returns a dictionary with the mean and standard deviation of the sample. 

In [138]:
def mean_and_std(i):
    """
    This function takes an input i and sets the random seed to i,
    which enables us to generate the same random sample everytime the
    seed is set to i. 
    """
    # setting the random seed
    np.random.seed(i)
    
    # generating a sample 
    random_numbers = np.random.normal(loc = 0, scale = 1, size = 1000000)
    
    # returning mean and standard deviation as a dictionary
    
    return {"mean": np.mean(random_numbers), "standard deviation": np.std(random_numbers)}
    

Now we will evaluate the above function 1000 times by repeatedly setting a different random seed and will append the results at the end of each iteration to an empty list that we initialize. We will also calculate how long it takes to evaluate this code so that we can compare timings when we run the code in parallel. 

In [139]:
# let's initialize an empty list where we will store the results from the mean_and_std function
collect = []

# starting time
start = time.time()

# let's run the mean_and_std function 100 times
# and collect the results in the empty list we initialized above

for i in range(1000):
    results = mean_and_std(i)
    collect.append(results)

# ending time
end = time.time()

# seconds it took to run the code in this cell
duration_non_parallelized = (end -start)

print("it took {} seconds to run the non parallelized code".format((duration_non_parallelized)))

it took 61.8604450226 seconds to run the non parallelized code


In [140]:
# let's seethe first 10 items in the list collect.
collect[:10]

[{'mean': 0.0015121465155362318, 'standard deviation': 0.9999213260166574},
 {'mean': 0.0006518043080192342, 'standard deviation': 0.9994898471333155},
 {'mean': -0.000977212317921356, 'standard deviation': 0.9992132599474451},
 {'mean': 0.001958404534987673, 'standard deviation': 0.9982829105321189},
 {'mean': -0.002798006780423977, 'standard deviation': 0.9997405575156038},
 {'mean': -2.0518082899011e-05, 'standard deviation': 0.9998743972589345},
 {'mean': 0.0003541923054206616, 'standard deviation': 0.9994563047066701},
 {'mean': -0.0004453479991014313, 'standard deviation': 1.0008483155135974},
 {'mean': 0.001307190140838998, 'standard deviation': 1.000003326371902},
 {'mean': -0.0017618465396848765, 'standard deviation': 1.0004889434552395}]

Till now whatever we did was only using 1 cpu in our laptop/deskptop.

If our laptop has more than 1 core which most modern laptops do, we can greatly reduce the time it takes to run the above code.

What it means is that, if we are running a for loop 1000 times and are only using 1 core to do it, the loop for the index `1` runs only after the loop has been evaluated for the index `0` and so on. However if we have more than 1 core then the loop can be evaluated simultaneously for the range of indexes we are running it for, leading to a reduction in the running time.  

In [141]:
# let's detect number of cpus
number_of_cpus = multiprocessing.cpu_count()

# print the number of cpus

print number_of_cpus

20


Running the code in parallel is pretty straighforward. We just need to use the object **Parallel** and function **delayed** that we imported at the start of this notebook form the **joblib** library. In short, within the **Parallel** object we need to define over how many cores we would like to evaluate our function and in the **delayed** function we pass the function name with the index and the range over which we would like to evaluate that function.     

In [142]:
# starting time
start = time.time()

# run the function in parallel and store the results in a list
collect = (Parallel(n_jobs=20)(delayed(mean_and_std)(i) for i in range(1000)))

# ending time
end = time.time()

# seconds it took to run the code in this cell
duration_parallelized = (end - start)

print("it took {} seconds to run the parallelized code".format((duration_parallelized)))


it took 4.40923810005 seconds to run the parallelized code


One key thing to note is that the output from running the code in parallel as above will be in the form of a list. So the 100 sets of dictionaries of means and standard deviations will all be in a list. We don't have to do initialize an empty list and append the dictionary of results at the end of each iteration like we did in the case of non parallelized code. 

In [143]:
# how much improvement did we make ? 

times = ((duration_non_parallelized   - duration_parallelized)/ duration_parallelized)

# print the improvement

times

13.029735663822542

The code that we ran in parallel is 13 times faster than the code we ran on a single core. This could mean the difference between a code that runs for a whole day as against a code that runs for only 4 hours. The time saved might not seem much in the example we used in this notebook since it was fairly simple and didn't take much time to run even on a single core. However the more complex a task is the more is the benefit derived from parallelizing the code. 


One question that I always had when I first started parallelizing my code was that whether the order of the indexes will be preserved in the final output. Since we had set a random seed within our function, this will be easy to verify. If everything worked out perfectly while running the code in parallel then the order of the results in the list from the parallelized code should be similar to the order of the results in the list from the non parallelized code. 

In [144]:
collect[:10]

[{'mean': 0.0015121465155362318, 'standard deviation': 0.9999213260166574},
 {'mean': 0.0006518043080192342, 'standard deviation': 0.9994898471333155},
 {'mean': -0.000977212317921356, 'standard deviation': 0.9992132599474451},
 {'mean': 0.001958404534987673, 'standard deviation': 0.9982829105321189},
 {'mean': -0.002798006780423977, 'standard deviation': 0.9997405575156038},
 {'mean': -2.0518082899011e-05, 'standard deviation': 0.9998743972589345},
 {'mean': 0.0003541923054206616, 'standard deviation': 0.9994563047066701},
 {'mean': -0.0004453479991014313, 'standard deviation': 1.0008483155135974},
 {'mean': 0.001307190140838998, 'standard deviation': 1.000003326371902},
 {'mean': -0.0017618465396848765, 'standard deviation': 1.0004889434552395}]

We see that the first `10` sets of means and standard deviations from the list of results in the case of parallelized code is exactly same as that in the case of non parallelized code.  

Therefore from this very simple example we see that parallelizing your code can greatly reduce the running time. This can particularly come in handy during kaggle competitions or other machine learning projects where you have to do a vast grid search for hyper parameter tuning in order to arrive at the best set of hyper parameters for your model. 